# Logistic industry Price Prediction

Varun Muralidharan vxm180016
Parthasarathy Sundar Karthikeyan PXS180018

# This data set is obtained from Kaggle. https://www.kaggle.com/divyeshardeshana/supply-chain-shipment-pricing-data

The data consist of export and logistic details of the medical industries from various countries and manufacturers or vendors. 
It allows to give an estimate of the price based on weight, insurance, shipment mode and several other factors which affect the cost.

In [0]:
import pandas as  pd
import numpy as np
import pandas_profiling
import matplotlib.pyplot as plt

In [0]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [126]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [127]:
data = pd.read_csv("/content/drive/My Drive/SCMS_Delivery_History_Dataset.csv")
data.head()

ID Project Code  ... Freight Cost (USD) Line Item Insurance (USD)
0   1   100-CI-T01  ...             780.34                       NaN
1   3   108-VN-T01  ...             4521.5                       NaN
2   4   100-CI-T01  ...            1653.78                       NaN
3  15   108-VN-T01  ...           16007.06                       NaN
4  16   108-VN-T01  ...           45450.08                       NaN

[5 rows x 33 columns]

In [128]:
# shape of Data
data.shape

(10324, 33)

In [129]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10324 entries, 0 to 10323
Data columns (total 33 columns):
ID                              10324 non-null int64
Project Code                    10324 non-null object
PQ #                            10324 non-null object
PO / SO #                       10324 non-null object
ASN/DN #                        10324 non-null object
Country                         10324 non-null object
Managed By                      10324 non-null object
Fulfill Via                     10324 non-null object
Vendor INCO Term                10324 non-null object
Shipment Mode                   9964 non-null object
PQ First Sent to Client Date    10324 non-null object
PO Sent to Vendor Date          10324 non-null object
Scheduled Delivery Date         10324 non-null object
Delivered to Client Date        10324 non-null object
Delivery Recorded Date          10324 non-null object
Product Group                   10324 non-null object
Sub Classification           

In [130]:
profile = pandas_profiling.ProfileReport(data, check_recoded = False)
display(profile)

/usr/local/lib/python3.6/dist-packages/pandas_profiling/describe.py:392: FutureWarning: The join_axes-keyword is deprecated. Use .reindex or .reindex_like on the result to achieve the same functionality.
  variable_stats = pd.concat(ldesc, join_axes=pd.Index([names]), axis=1)


In [131]:
# Missing values in data
data.isna().sum()

ID                                 0
Project Code                       0
PQ #                               0
PO / SO #                          0
ASN/DN #                           0
Country                            0
Managed By                         0
Fulfill Via                        0
Vendor INCO Term                   0
Shipment Mode                    360
PQ First Sent to Client Date       0
PO Sent to Vendor Date             0
Scheduled Delivery Date            0
Delivered to Client Date           0
Delivery Recorded Date             0
Product Group                      0
Sub Classification                 0
Vendor                             0
Item Description                   0
Molecule/Test Type                 0
Brand                              0
Dosage                          1736
Dosage Form                        0
Unit of Measure (Per Pack)         0
Line Item Quantity                 0
Line Item Value                    0
Pack Price                         0
U

In [0]:
# Imputing Line Item Insurance (USD) with 0 since the transaction did not have insurance cost

data['Line Item Insurance (USD)']=data['Line Item Insurance (USD)'].fillna(0)

In [133]:
# Rechecking

data.isna().sum()

ID                                 0
Project Code                       0
PQ #                               0
PO / SO #                          0
ASN/DN #                           0
Country                            0
Managed By                         0
Fulfill Via                        0
Vendor INCO Term                   0
Shipment Mode                    360
PQ First Sent to Client Date       0
PO Sent to Vendor Date             0
Scheduled Delivery Date            0
Delivered to Client Date           0
Delivery Recorded Date             0
Product Group                      0
Sub Classification                 0
Vendor                             0
Item Description                   0
Molecule/Test Type                 0
Brand                              0
Dosage                          1736
Dosage Form                        0
Unit of Measure (Per Pack)         0
Line Item Quantity                 0
Line Item Value                    0
Pack Price                         0
U

In [134]:
# Shipment mode has some NaN values so we replace with no ship
data['Shipment Mode']=data['Shipment Mode'].fillna('no ship')
data.isna().sum()

ID                                 0
Project Code                       0
PQ #                               0
PO / SO #                          0
ASN/DN #                           0
Country                            0
Managed By                         0
Fulfill Via                        0
Vendor INCO Term                   0
Shipment Mode                      0
PQ First Sent to Client Date       0
PO Sent to Vendor Date             0
Scheduled Delivery Date            0
Delivered to Client Date           0
Delivery Recorded Date             0
Product Group                      0
Sub Classification                 0
Vendor                             0
Item Description                   0
Molecule/Test Type                 0
Brand                              0
Dosage                          1736
Dosage Form                        0
Unit of Measure (Per Pack)         0
Line Item Quantity                 0
Line Item Value                    0
Pack Price                         0
U

In [135]:
# Dosage has some NaN values so we replace with no dosage
data['Dosage']=data['Dosage'].fillna('no dosage')
data.isna().sum()

ID                              0
Project Code                    0
PQ #                            0
PO / SO #                       0
ASN/DN #                        0
Country                         0
Managed By                      0
Fulfill Via                     0
Vendor INCO Term                0
Shipment Mode                   0
PQ First Sent to Client Date    0
PO Sent to Vendor Date          0
Scheduled Delivery Date         0
Delivered to Client Date        0
Delivery Recorded Date          0
Product Group                   0
Sub Classification              0
Vendor                          0
Item Description                0
Molecule/Test Type              0
Brand                           0
Dosage                          0
Dosage Form                     0
Unit of Measure (Per Pack)      0
Line Item Quantity              0
Line Item Value                 0
Pack Price                      0
Unit Price                      0
Manufacturing Site              0
First Line Des

In [136]:
data.shape

(10324, 33)

In [0]:
# replacing reference 'Freight Cost (USD)'
for i in range(0,10324):
    if 'See' in data.iloc[i,31]:
        p=int(data.iloc[i,31].split(':')[1].split(')')[0])
        m=data.loc[data['ID'] == p, 'Freight Cost (USD)'].iloc[0]
        data.iloc[i,31]=m

In [138]:
# Checking
data['Freight Cost (USD)'].head(15)

0                                 780.34
1                                 4521.5
2                                1653.78
3                               16007.06
4                               45450.08
5                                5920.42
6     Freight Included in Commodity Cost
7                                6212.41
8                                4861.14
9     Freight Included in Commodity Cost
10    Freight Included in Commodity Cost
11                              13569.49
12                   Invoiced Separately
13                              64179.42
14                               1760.32
Name: Freight Cost (USD), dtype: object

In [139]:
data['Scheduled Delivery Date'].head()

0     2-Jun-06
1    14-Nov-06
2    27-Aug-06
3     1-Sep-06
4    11-Aug-06
Name: Scheduled Delivery Date, dtype: object

In [140]:
data.iloc[8,31]

'4861.14'

In [0]:
# converting to datetime
data['Scheduled Delivery Date']=pd.to_datetime(data['Scheduled Delivery Date'])

In [0]:
data['Delivered to Client Date']=pd.to_datetime(data['Delivered to Client Date'])
data['Delivery Recorded Date']=pd.to_datetime(data['Delivery Recorded Date'])

In [0]:
# Calculating delay ..... + means delay -  means early
data['delay']=data['Delivery Recorded Date']-data['Scheduled Delivery Date']

In [144]:
data.shape

(10324, 34)

In [145]:
data.iloc[:,30].head(10)


0                       13
1                      358
2                      171
3                     1855
4                     7590
5                      504
6                      328
7                     1478
8    See ASN-93 (ID#:1281)
9                      643
Name: Weight (Kilograms), dtype: object

In [0]:
for i in range(0,10324):
    if 'See' in data.iloc[i,30]:
        p=int(data.iloc[i,30].split(':')[1].split(')')[0])
        m=data.loc[data['ID'] == p, 'Weight (Kilograms)'].iloc[0]
        data.iloc[i,30]=m

In [147]:
data['Weight (Kilograms)'].head(15)

0                             13
1                            358
2                            171
3                           1855
4                           7590
5                            504
6                            328
7                           1478
8                            479
9                            643
10                           643
11                           853
12    Weight Captured Separately
13                          7416
14                           256
Name: Weight (Kilograms), dtype: object

In [148]:
data.iloc[6,31]

'Freight Included in Commodity Cost'

In [149]:
# Checking how many Freight cost has "Freight Included in Commodity Cost"
search="Freight Included in Commodity Cost"
x=data["Freight Cost (USD)"].str.count(search)
x.sum()

1442

In [0]:
# Removing them
data=data[data["Freight Cost (USD)"]!="Freight Included in Commodity Cost"]


In [151]:
data.shape

(8882, 34)

In [152]:
# Removing Seperately invoiced rows
search="Invoiced Separately"
x=data["Freight Cost (USD)"].str.count(search)
x.sum()

data=data[data["Freight Cost (USD)"]!="Invoiced Separately"]

344

In [153]:
data.shape

(8538, 34)

In [154]:
# Removing "Weight Captured Separately" from Weights Column
search="Weight Captured Separately"
x=data["Weight (Kilograms)"].str.count(search)


data=data[data["Weight (Kilograms)"]!="Weight Captured Separately"]
data.shape

(8493, 34)

In [155]:
x.sum()

45

In [0]:
# Pre Processing 

In [0]:
data["Weight (Kilograms)"]=data["Weight (Kilograms)"].astype('int32')

In [158]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8493 entries, 0 to 10320
Data columns (total 34 columns):
ID                              8493 non-null int64
Project Code                    8493 non-null object
PQ #                            8493 non-null object
PO / SO #                       8493 non-null object
ASN/DN #                        8493 non-null object
Country                         8493 non-null object
Managed By                      8493 non-null object
Fulfill Via                     8493 non-null object
Vendor INCO Term                8493 non-null object
Shipment Mode                   8493 non-null object
PQ First Sent to Client Date    8493 non-null object
PO Sent to Vendor Date          8493 non-null object
Scheduled Delivery Date         8493 non-null datetime64[ns]
Delivered to Client Date        8493 non-null datetime64[ns]
Delivery Recorded Date          8493 non-null datetime64[ns]
Product Group                   8493 non-null object
Sub Classification   

In [0]:
#data["Freight Cost (USD)"]=data["Freight Cost (USD)"].astype('int32')

In [0]:
data["Freight Cost (USD)"]=pd.to_numeric(data["Freight Cost (USD)"])

In [161]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8493 entries, 0 to 10320
Data columns (total 34 columns):
ID                              8493 non-null int64
Project Code                    8493 non-null object
PQ #                            8493 non-null object
PO / SO #                       8493 non-null object
ASN/DN #                        8493 non-null object
Country                         8493 non-null object
Managed By                      8493 non-null object
Fulfill Via                     8493 non-null object
Vendor INCO Term                8493 non-null object
Shipment Mode                   8493 non-null object
PQ First Sent to Client Date    8493 non-null object
PO Sent to Vendor Date          8493 non-null object
Scheduled Delivery Date         8493 non-null datetime64[ns]
Delivered to Client Date        8493 non-null datetime64[ns]
Delivery Recorded Date          8493 non-null datetime64[ns]
Product Group                   8493 non-null object
Sub Classification   

In [0]:
# Dropping Irrelevant columns, most of the columns are ids or dates which have been transformed to another variable. Some Variables were 
# also repeating which might cause the model to fail.
df=data
df.drop(df.columns[[1,2,3,4,10,11,12,13,14,18,19]], axis=1, inplace=True)

In [163]:
df.columns

Index(['ID', 'Country', 'Managed By', 'Fulfill Via', 'Vendor INCO Term',
       'Shipment Mode', 'Product Group', 'Sub Classification', 'Vendor',
       'Brand', 'Dosage', 'Dosage Form', 'Unit of Measure (Per Pack)',
       'Line Item Quantity', 'Line Item Value', 'Pack Price', 'Unit Price',
       'Manufacturing Site', 'First Line Designation', 'Weight (Kilograms)',
       'Freight Cost (USD)', 'Line Item Insurance (USD)', 'delay'],
      dtype='object')

In [0]:
data['delay_or early']=(df['delay'] / np.timedelta64(1, 'D')).astype(int)

In [0]:
df['delay_or early']=(df['delay'] / np.timedelta64(1, 'D')).astype(int)

In [166]:
df.shape

(8493, 24)

In [0]:
df = df.drop(['delay'], axis=1)

In [168]:
df.columns

Index(['ID', 'Country', 'Managed By', 'Fulfill Via', 'Vendor INCO Term',
       'Shipment Mode', 'Product Group', 'Sub Classification', 'Vendor',
       'Brand', 'Dosage', 'Dosage Form', 'Unit of Measure (Per Pack)',
       'Line Item Quantity', 'Line Item Value', 'Pack Price', 'Unit Price',
       'Manufacturing Site', 'First Line Designation', 'Weight (Kilograms)',
       'Freight Cost (USD)', 'Line Item Insurance (USD)', 'delay_or early'],
      dtype='object')

In [0]:
df = df.drop(['Manufacturing Site'], axis=1)

In [170]:
df.columns

Index(['ID', 'Country', 'Managed By', 'Fulfill Via', 'Vendor INCO Term',
       'Shipment Mode', 'Product Group', 'Sub Classification', 'Vendor',
       'Brand', 'Dosage', 'Dosage Form', 'Unit of Measure (Per Pack)',
       'Line Item Quantity', 'Line Item Value', 'Pack Price', 'Unit Price',
       'First Line Designation', 'Weight (Kilograms)', 'Freight Cost (USD)',
       'Line Item Insurance (USD)', 'delay_or early'],
      dtype='object')

In [0]:
# Dropping ID since it doesnt help in prediction
df=df.drop(["ID"],axis=1)



In [172]:
df=df.drop(["ones"], axis = 1)

KeyError: ignored

In [173]:
df.columns

Index(['Country', 'Managed By', 'Fulfill Via', 'Vendor INCO Term',
       'Shipment Mode', 'Product Group', 'Sub Classification', 'Vendor',
       'Brand', 'Dosage', 'Dosage Form', 'Unit of Measure (Per Pack)',
       'Line Item Quantity', 'Line Item Value', 'Pack Price', 'Unit Price',
       'First Line Designation', 'Weight (Kilograms)', 'Freight Cost (USD)',
       'Line Item Insurance (USD)', 'delay_or early'],
      dtype='object')

In [174]:
df['Country'].value_counts()

# Removing all countries with exports less than 10

Côte d'Ivoire         1047
Nigeria               1020
Uganda                 752
Zambia                 650
Vietnam                643
Haiti                  596
Mozambique             569
Zimbabwe               518
Tanzania               491
Rwanda                 397
Congo, DRC             292
South Africa           288
Guyana                 227
Ethiopia               204
South Sudan            130
Kenya                   93
Burundi                 92
Namibia                 90
Cameroon                66
Botswana                64
Ghana                   48
Dominican Republic      45
Sudan                   42
Swaziland               33
Mali                    14
Malawi                  14
Pakistan                13
Benin                   12
Guatemala               10
Libya                    8
Angola                   7
Liberia                  6
Afghanistan              3
Sierra Leone             2
Senegal                  2
Togo                     2
Lesotho                  2
G

In [175]:
df.shape

(8493, 21)

In [176]:
df2

Unit of Measure (Per Pack)  ...  First Line Designation_Yes
0                              30  ...                           1
1                             240  ...                           1
2                             100  ...                           1
3                              60  ...                           1
4                              60  ...                           1
...                           ...  ...                         ...
10316                          60  ...                           1
10317                          30  ...                           0
10318                          60  ...                           0
10319                          60  ...                           0
10320                          60  ...                           0

[8460 rows x 127 columns]

In [177]:
df2=df[df['Country']!='Guinea']
df2.shape

(8492, 21)

In [178]:
df2=df2[df2['Country']!='Sierra Leone']
df2=df2[df2['Country']!='Lesotho']
df2=df2[df2['Country']!='Togo']
df2=df2[df2['Country']!='Senegal']
df2=df2[df2['Country']!='Afghanistan']
df2=df2[df2['Country']!='Liberia']
df2=df2[df2['Country']!='Angola']
df2=df2[df2['Country']!='Libya']
df2.shape

(8460, 21)

In [179]:
df2['Country'].nunique()

29

In [180]:
df2.columns
df2['Managed By'].value_counts()

Index(['Country', 'Managed By', 'Fulfill Via', 'Vendor INCO Term',
       'Shipment Mode', 'Product Group', 'Sub Classification', 'Vendor',
       'Brand', 'Dosage', 'Dosage Form', 'Unit of Measure (Per Pack)',
       'Line Item Quantity', 'Line Item Value', 'Pack Price', 'Unit Price',
       'First Line Designation', 'Weight (Kilograms)', 'Freight Cost (USD)',
       'Line Item Insurance (USD)', 'delay_or early'],
      dtype='object')

PMO - US              8459
Haiti Field Office       1
Name: Managed By, dtype: int64

In [181]:
df2.columns
df2['Vendor INCO Term'].value_counts()

Index(['Country', 'Managed By', 'Fulfill Via', 'Vendor INCO Term',
       'Shipment Mode', 'Product Group', 'Sub Classification', 'Vendor',
       'Brand', 'Dosage', 'Dosage Form', 'Unit of Measure (Per Pack)',
       'Line Item Quantity', 'Line Item Value', 'Pack Price', 'Unit Price',
       'First Line Designation', 'Weight (Kilograms)', 'Freight Cost (USD)',
       'Line Item Insurance (USD)', 'delay_or early'],
      dtype='object')

N/A - From RDC    5051
EXW               2553
FCA                366
DDP                303
CIP                178
DAP                  5
CIF                  2
DDU                  2
Name: Vendor INCO Term, dtype: int64

In [182]:
df2[df2['Vendor INCO Term']=='CIF']

Country Managed By  ... Line Item Insurance (USD) delay_or early
681     Guyana   PMO - US  ...                      23.4              0
3264  Zimbabwe   PMO - US  ...                      38.6              0

[2 rows x 21 columns]

In [183]:
df2.columns
df2['Vendor'].value_counts()
df2['Vendor'].nunique()

Index(['Country', 'Managed By', 'Fulfill Via', 'Vendor INCO Term',
       'Shipment Mode', 'Product Group', 'Sub Classification', 'Vendor',
       'Brand', 'Dosage', 'Dosage Form', 'Unit of Measure (Per Pack)',
       'Line Item Quantity', 'Line Item Value', 'Pack Price', 'Unit Price',
       'First Line Designation', 'Weight (Kilograms)', 'Freight Cost (USD)',
       'Line Item Insurance (USD)', 'delay_or early'],
      dtype='object')

SCMS from RDC                                                        5051
Orgenics, Ltd                                                         642
Aurobindo Pharma Limited                                              626
Trinity Biotech, Plc                                                  336
ABBVIE LOGISTICS (FORMERLY ABBOTT LOGISTICS BV)                       316
MYLAN LABORATORIES LTD (FORMERLY MATRIX LABORATORIES)                 304
HETERO LABS LIMITED                                                   246
CIPLA LIMITED                                                         156
CHEMBIO DIAGNOSTIC SYSTEMS, INC.                                      103
Standard Diagnostics, Inc.                                             94
STRIDES ARCOLAB LIMITED                                                90
SHANGHAI KEHUA BIOENGINEERING CO.,LTD.  (KHB)                          70
MERCK SHARP & DOHME IDEA GMBH (FORMALLY MERCK SHARP & DOHME B.V.)      50
Orasure Technologies Inc.             

52

In [184]:
df2.columns
df2['Dosage'].value_counts()
# DRop dosage columns
df2 = df2.drop(['Dosage'], axis=1)

Index(['Country', 'Managed By', 'Fulfill Via', 'Vendor INCO Term',
       'Shipment Mode', 'Product Group', 'Sub Classification', 'Vendor',
       'Brand', 'Dosage', 'Dosage Form', 'Unit of Measure (Per Pack)',
       'Line Item Quantity', 'Line Item Value', 'Pack Price', 'Unit Price',
       'First Line Designation', 'Weight (Kilograms)', 'Freight Cost (USD)',
       'Line Item Insurance (USD)', 'delay_or early'],
      dtype='object')

no dosage          1513
300mg               762
200mg               753
600mg               639
150/300/200mg       531
150/300mg           527
10mg/ml             433
200/50mg            335
150mg               333
300/300mg           288
600/300/300mg       254
150/200/30mg        235
100mg               162
150/30mg            130
600/200/300mg       120
30/50/60mg          118
200/300mg           115
30mg                109
50mg                108
20mg/ml             101
80/20mg/ml           94
30/60mg              82
300/200mg            76
400mg                73
60/30mg              65
100/25mg             65
600/300mg            60
250mg                45
300/100mg            44
15mg                 34
1mg/ml               31
30mg/ml              30
25mg                 25
300/150/300mg        24
20mg                 24
30/50/6mg            19
30/6mg               13
60/100/12mg          12
500/25mg             12
20/120mg             12
500mg                 9
150/300+200mg   

In [185]:
df2.columns
#df2['Brand'].value_counts()
#df2['Brand'].nunique()
df2.shape

Index(['Country', 'Managed By', 'Fulfill Via', 'Vendor INCO Term',
       'Shipment Mode', 'Product Group', 'Sub Classification', 'Vendor',
       'Brand', 'Dosage Form', 'Unit of Measure (Per Pack)',
       'Line Item Quantity', 'Line Item Value', 'Pack Price', 'Unit Price',
       'First Line Designation', 'Weight (Kilograms)', 'Freight Cost (USD)',
       'Line Item Insurance (USD)', 'delay_or early'],
      dtype='object')

(8460, 20)

In [0]:
# Feature Transformation
# Df2 is transformed

In [0]:
data1=df2
#df2=data1
#df2.columns

In [188]:
data1.shape

(8460, 20)

In [0]:
# One hot encoding of countries and avoiding dummy trap
# Other Nominal variables are 'Managed By', 'Fulfill Via', 'Vendor INCO Term', 'Shipment Mode',
#      'Product Group', 'Sub Classification', 'Vendor', 'Brand', 'Dosage Form', 'First Line Designation'
# Needs oene hot encoding

In [190]:
#df2.columns
df2Dummies2 = pd.get_dummies(df2["Country"], prefix = "Country")
v=df2Dummies2.drop(df2Dummies2.columns[0], axis=1)
df2 = pd.concat([df2, v], axis = 1)
df2.head()
df2=df2.drop(["Country"],axis=1)


Country Managed By  ... Country_Zambia Country_Zimbabwe
0  Côte d'Ivoire   PMO - US  ...              0                0
1        Vietnam   PMO - US  ...              0                0
2  Côte d'Ivoire   PMO - US  ...              0                0
3        Vietnam   PMO - US  ...              0                0
4        Vietnam   PMO - US  ...              0                0

[5 rows x 48 columns]

In [191]:
v.shape

(8460, 28)

In [192]:
df2.columns

Index(['Managed By', 'Fulfill Via', 'Vendor INCO Term', 'Shipment Mode',
       'Product Group', 'Sub Classification', 'Vendor', 'Brand', 'Dosage Form',
       'Unit of Measure (Per Pack)', 'Line Item Quantity', 'Line Item Value',
       'Pack Price', 'Unit Price', 'First Line Designation',
       'Weight (Kilograms)', 'Freight Cost (USD)', 'Line Item Insurance (USD)',
       'delay_or early', 'Country_Botswana', 'Country_Burundi',
       'Country_Cameroon', 'Country_Congo, DRC', 'Country_Côte d'Ivoire',
       'Country_Dominican Republic', 'Country_Ethiopia', 'Country_Ghana',
       'Country_Guatemala', 'Country_Guyana', 'Country_Haiti', 'Country_Kenya',
       'Country_Malawi', 'Country_Mali', 'Country_Mozambique',
       'Country_Namibia', 'Country_Nigeria', 'Country_Pakistan',
       'Country_Rwanda', 'Country_South Africa', 'Country_South Sudan',
       'Country_Sudan', 'Country_Swaziland', 'Country_Tanzania',
       'Country_Uganda', 'Country_Vietnam', 'Country_Zambia',
       'C

In [193]:
# one hot encoding for Managed by , 0 means Haiti office. Since Nominal variable
df2Dummies2 = pd.get_dummies(df2["Managed By"], prefix = "Managed By")
v=df2Dummies2.drop(df2Dummies2.columns[0], axis=1)
df2 = pd.concat([df2, v], axis = 1)
df2.head()
df2=df2.drop(["Managed By"],axis=1)

Managed By  Fulfill Via  ... Country_Zimbabwe Managed By_PMO - US
0   PMO - US  Direct Drop  ...                0                   1
1   PMO - US  Direct Drop  ...                0                   1
2   PMO - US  Direct Drop  ...                0                   1
3   PMO - US  Direct Drop  ...                0                   1
4   PMO - US  Direct Drop  ...                0                   1

[5 rows x 48 columns]

In [0]:
# one hot encoding for 'Fulfill Via' , 0 means from RDC,  1 means direct drop, Since Nominal variable
df2Dummies2 = pd.get_dummies(df2['Fulfill Via'], prefix = 'Fulfill Via')
v=df2Dummies2.drop(df2Dummies2.columns[0], axis=1)
df2 = pd.concat([df2, v], axis = 1)
df2=df2.drop(['Fulfill Via'],axis=1)

In [0]:
# one hot encoding for 'Vendor INCO Term' , CIF removed for dummy trap, Since Nominal variable
df2Dummies2 = pd.get_dummies(df2['Vendor INCO Term'], prefix = 'Vendor INCO Term')
v=df2Dummies2.drop(df2Dummies2.columns[0], axis=1)
df2 = pd.concat([df2, v], axis = 1)
df2=df2.drop(['Vendor INCO Term'],axis=1)

In [0]:
# one hot encoding for 'Shipment Mode' , AIR removed for dummy trap, Since Nominal variable
df2Dummies2 = pd.get_dummies(df2['Shipment Mode'], prefix = 'Shipment Mode')
v=df2Dummies2.drop(df2Dummies2.columns[0], axis=1)
df2 = pd.concat([df2, v], axis = 1)
df2=df2.drop(['Shipment Mode'],axis=1)

In [197]:
# one hot encoding for 'Product Group' , ACT removed for dummy trap, Since Nominal variable
df2Dummies2 = pd.get_dummies(df2['Product Group'], prefix = 'Product Group')
v=df2Dummies2.drop(df2Dummies2.columns[0], axis=1)
df2 = pd.concat([df2, v], axis = 1)
df2=df2.drop(['Product Group'],axis=1)
df2.head()

Sub Classification  ... Product Group_MRDT
0           HIV test  ...                  0
1          Pediatric  ...                  0
2           HIV test  ...                  0
3              Adult  ...                  0
4              Adult  ...                  0

[5 rows x 59 columns]

In [198]:
# one hot encoding for 'Sub Classification' , ACT removed for dummy trap
df2Dummies2 = pd.get_dummies(df2['Sub Classification'], prefix = 'Sub Classification')
v=df2Dummies2.drop(df2Dummies2.columns[0], axis=1)
df2 = pd.concat([df2, v], axis = 1)
df2=df2.drop(['Sub Classification'],axis=1)
df2.head()

Vendor  ... Sub Classification_Pediatric
0                        RANBAXY Fine Chemicals LTD.  ...                            0
1                           Aurobindo Pharma Limited  ...                            1
2                               Abbott GmbH & Co. KG  ...                            0
3  SUN PHARMACEUTICAL INDUSTRIES LTD (RANBAXY LAB...  ...                            0
4                           Aurobindo Pharma Limited  ...                            0

[5 rows x 63 columns]

In [199]:
# one hot encoding for 'Vendor' , Abbott GmbH & Co. KG removed for dummy trap
df2Dummies2 = pd.get_dummies(df2['Vendor'], prefix = 'Vendor')
v=df2Dummies2.drop(df2Dummies2.columns[0], axis=1)
df2 = pd.concat([df2, v], axis = 1)
df2=df2.drop(['Vendor'],axis=1)
df2.head()

Brand  ... Vendor_ZEPHYR BIOMEDICALS
0     Reveal  ...                         0
1    Generic  ...                         0
2  Determine  ...                         0
3    Generic  ...                         0
4    Generic  ...                         0

[5 rows x 113 columns]

In [200]:
# Dropping Brand
df2=df2.drop(['Brand'],axis=1)
df2.head()

Dosage Form  ...  Vendor_ZEPHYR BIOMEDICALS
0         Test kit  ...                          0
1  Oral suspension  ...                          0
2         Test kit  ...                          0
3           Tablet  ...                          0
4          Capsule  ...                          0

[5 rows x 112 columns]

In [201]:
# one hot encoding for 'Dosage Form', Capsule removed for dummy trap
df2Dummies2 = pd.get_dummies(df2['Dosage Form'], prefix = 'Dosage Form')
v=df2Dummies2.drop(df2Dummies2.columns[0], axis=1)
df2 = pd.concat([df2, v], axis = 1)
df2=df2.drop(['Dosage Form'],axis=1)
df2.head()

Unit of Measure (Per Pack)  ...  Dosage Form_Test kit - Ancillary
0                          30  ...                                 0
1                         240  ...                                 0
2                         100  ...                                 0
3                          60  ...                                 0
4                          60  ...                                 0

[5 rows x 127 columns]

In [202]:
# one hot encoding for 'First Line Designation, Yes = 1 dummy trap
df2Dummies2 = pd.get_dummies(df2['First Line Designation'], prefix = 'First Line Designation')
v=df2Dummies2.drop(df2Dummies2.columns[0], axis=1)
df2 = pd.concat([df2, v], axis = 1)
df2=df2.drop(['First Line Designation'],axis=1)
df2.head()

Unit of Measure (Per Pack)  ...  First Line Designation_Yes
0                          30  ...                           1
1                         240  ...                           1
2                         100  ...                           1
3                          60  ...                           1
4                          60  ...                           1

[5 rows x 127 columns]

In [203]:
df2.columns
df2.shape

Index(['Unit of Measure (Per Pack)', 'Line Item Quantity', 'Line Item Value',
       'Pack Price', 'Unit Price', 'Weight (Kilograms)', 'Freight Cost (USD)',
       'Line Item Insurance (USD)', 'delay_or early', 'Country_Botswana',
       ...
       'Dosage Form_Oral suspension', 'Dosage Form_Powder for oral solution',
       'Dosage Form_Tablet', 'Dosage Form_Tablet - FDC',
       'Dosage Form_Tablet - FDC + blister',
       'Dosage Form_Tablet - FDC + co-blister', 'Dosage Form_Tablet - blister',
       'Dosage Form_Test kit', 'Dosage Form_Test kit - Ancillary',
       'First Line Designation_Yes'],
      dtype='object', length=127)

(8460, 127)

In [0]:
df3=df2

In [0]:
# End of Cleaning and Filtering

In [0]:
v=df3.columns

# Splitting X vs y from data

In [207]:
#Split into X and y

X= df2.drop(['Freight Cost (USD)'], axis=1)
y=df2[['Freight Cost (USD)']]
X.shape

(8460, 126)

In [0]:
#X=X.drop(['ones'], axis=1)

In [0]:
#X=X.drop(['ID'], axis=1)


In [209]:
y.head()


Freight Cost (USD)
0              780.34
1             4521.50
2             1653.78
3            16007.06
4            45450.08

# We are using train test split to get the train and test set.

In [0]:
from sklearn.model_selection import train_test_split

X_train_org, X_test_org, y_train, y_test = train_test_split(X,y,random_state = 0)


# We are using standard Scaler to scale the variables such that we can use the data for distance based algorithms.

In [0]:

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train_org)
X_test = scaler.transform(X_test_org)

In [0]:
data=data.drop(['ID'], axis=1)

In [0]:
features=data.columns

In [214]:
data.shape

(8493, 23)

In [215]:
features[12:24]

Index(['Line Item Quantity', 'Line Item Value', 'Pack Price', 'Unit Price',
       'Manufacturing Site', 'First Line Designation', 'Weight (Kilograms)',
       'Freight Cost (USD)', 'Line Item Insurance (USD)', 'delay',
       'delay_or early'],
      dtype='object')

# Project 1

## KNN Regressor

In [0]:
#from pandas.plotting import scatter_matrix
#attributes = features[12:24]
#scatter_matrix(X[attributes], figsize = (15,15), c = y, alpha = 0.8, marker = 'O')

In [217]:
from sklearn import neighbors
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import GridSearchCV
params = {'n_neighbors':[2,3,4,5,6,7,8,9,11,12,13,14,15,16]}
 
knn = neighbors.KNeighborsRegressor()
model = GridSearchCV(knn, params, cv=5)
model.fit(X_train,y_train)
model.best_params_


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=KNeighborsRegressor(algorithm='auto', leaf_size=30,
                                           metric='minkowski',
                                           metric_params=None, n_jobs=None,
                                           n_neighbors=5, p=2,
                                           weights='uniform'),
             iid='warn', n_jobs=None,
             param_grid={'n_neighbors': [2, 3, 4, 5, 6, 7, 8, 9, 11, 12, 13, 14,
                                         15, 16]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

{'n_neighbors': 2}

In [218]:
knn = neighbors.KNeighborsRegressor(2)
knn.fit(X_train,y_train)
y_pred_knn=knn.predict(X_test)
print("KNN train score ",knn.score(X_train, y_train))
#print("Mean squared error ",mean_squared_error(y_test,y_pred_knn))
print("KNN test score ",knn.score(X_test, y_test))



KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
                    metric_params=None, n_jobs=None, n_neighbors=2, p=2,
                    weights='uniform')

KNN train score  0.8839869407347604
KNN test score  0.5012804121698169


In [219]:
y_pred_knn.shape

(2115, 1)

In [220]:
y_test.shape

(2115, 1)

In [0]:
# conclusion: 
# Train and Test score is not good even with best parameters. So we can conclude that KNN REgressor is not a good model. 

## Linear model

In [0]:
from sklearn.metrics import r2_score 

In [0]:
#data1=data1.drop(['ones'], axis=1)

In [224]:
X.head()

Unit of Measure (Per Pack)  ...  First Line Designation_Yes
0                          30  ...                           1
1                         240  ...                           1
2                         100  ...                           1
3                          60  ...                           1
4                          60  ...                           1

[5 rows x 126 columns]

## Train test Spliting

In [0]:
#X= data1.drop(['Freight Cost (USD)'], axis=1)
#y=data1[['Freight Cost (USD)']]
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.20, random_state = 0)


In [226]:
X_train.head(1)

Unit of Measure (Per Pack)  ...  First Line Designation_Yes
291                          20  ...                           1

[1 rows x 126 columns]

In [227]:
X_test.head(1)

Unit of Measure (Per Pack)  ...  First Line Designation_Yes
1072                         100  ...                           1

[1 rows x 126 columns]

In [228]:
from sklearn.linear_model import LinearRegression

lreg = LinearRegression()
lreg.fit(X_train, y_train)
print(lreg.score(X_train, y_train))
print(lreg.score(X_test, y_test))

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

0.3075139361701459
0.1849668503446874


In [0]:
# Very poor model and is not generalizing well 

In [0]:
#plt.scatter(data1['Freight Cost (USD)'],data1['ID'])

## Ridge Regression

In [0]:


#x_range = [0.01, 0.1, 1, 10, 100, 1000, 10000, 20000]
#train_score_list = []
#test_score_list = []

#for alpha in x_range: 
#    ridge = Ridge(alpha)
#    ridge.fit(X_train,y_train)
#    train_score_list.append(ridge.score(X_train,y_train))
#    test_score_list.append(ridge.score(X_test, y_test))

In [232]:
from  sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV

alphas = np.array([1,0.1,0.01,0.001,0.0001,0])

model = Ridge()
grid = GridSearchCV(estimator=model, param_grid=dict(alpha=alphas))
grid.fit(X_train, y_train)
print(grid)
# summarize the results of the grid search
print(grid.best_score_)
print(grid.best_estimator_.alpha)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/ridge.py:147: LinAlgWarning: Ill-conditioned matrix (rcond=1.03143e-16): result may not be accurate.
  overwrite_a=True).T
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/ridge.py:147: LinAlgWarning: Ill-conditioned matrix (rcond=7.48379e-17): result may not be accurate.
  overwrite_a=True).T
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/ridge.py:147: LinAlgWarning: Ill-conditioned matrix (rcond=1.02101e-16): result may not be accurate.
  overwrite_a=True).T
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/ridge.py:147: LinAlgWarning: Ill-conditioned matrix (rcond=6.48268e-18): result may not be accurate.
  overwrite_a=True).T
/usr/local/lib/pyth

GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=Ridge(alpha=1.0, copy_X=True, fit_intercept=True,
                             max_iter=None, normalize=False, random_state=None,
                             solver='auto', tol=0.001),
             iid='warn', n_jobs=None,
             param_grid={'alpha': array([1.e+00, 1.e-01, 1.e-02, 1.e-03, 1.e-04, 0.e+00])},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=Ridge(alpha=1.0, copy_X=True, fit_intercept=True,
                             max_iter=None, normalize=False, random_state=None,
                             solver='auto', tol=0.001),
             iid='warn', n_jobs=None,
             param_grid={'alpha': array([1.e+00, 1.e-01, 1.e-02, 1.e-03, 1.e-04, 0.e+00])},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)
0.22291349937056493
1.0


In [233]:
ridge = Ridge(alpha=1.0)
ridge.fit(X_train,y_train)
print(ridge.score(X_train,y_train))
print(ridge.score(X_test,y_test))

Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
      normalize=False, random_state=None, solver='auto', tol=0.001)

0.30719181958406727
0.18138512503957782


In [0]:
# Not Good enough

## Lasso Regression

In [0]:

#x_range = [0.01, 0.1, 1, 10, 100,1000]
#train_score_list = []
#test_score_list = []

#for alpha in x_range: 
#   lasso = Lasso(alpha)
#    lasso.fit(X_train,y_train)
#   train_score_list.append(lasso.score(X_train,y_train))
#   test_score_list.append(lasso.score(X_test, y_test))

In [236]:
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV

alphas = np.array([1,0.1,0.01,0.001,0.0001,0])

model = Lasso()
grid = GridSearchCV(estimator=model, param_grid=dict(alpha=alphas))
grid.fit(X_train, y_train)
print(grid)
# summarize the results of the grid search
print(grid.best_score_)
print(grid.best_estimator_.alpha)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 488569954878.3966, tolerance: 155212282.10897934
  positive)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 557619099143.5946, tolerance: 160907777.44679028
  positive)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 500801410951.1354, toleran

GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=Lasso(alpha=1.0, copy_X=True, fit_intercept=True,
                             max_iter=1000, normalize=False, positive=False,
                             precompute=False, random_state=None,
                             selection='cyclic', tol=0.0001, warm_start=False),
             iid='warn', n_jobs=None,
             param_grid={'alpha': array([1.e+00, 1.e-01, 1.e-02, 1.e-03, 1.e-04, 0.e+00])},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=Lasso(alpha=1.0, copy_X=True, fit_intercept=True,
                             max_iter=1000, normalize=False, positive=False,
                             precompute=False, random_state=None,
                             selection='cyclic', tol=0.0001, warm_start=False),
             iid='warn', n_jobs=None,
             param_grid={'alpha': array([1.e+00, 1.e-01, 1.e-02, 1.e-03, 1.e-04, 0.e+00])},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)
0.2205547960546506
1.0


In [237]:
from sklearn.linear_model import Lasso
lasso = Lasso(alpha=1.0)
lasso.fit(X_train,y_train)
print(lasso.score(X_train,y_train))
print(lasso.score(X_test,y_test))

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 799635089006.8865, tolerance: 230287471.16720325
  positive)


Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=1000,
      normalize=False, positive=False, precompute=False, random_state=None,
      selection='cyclic', tol=0.0001, warm_start=False)

0.30712195913032425
0.17259434882381053


In [0]:
#Not Good Enough

In [0]:
from sklearn.linear_model import LinearRegression
lreg = LinearRegression()

## Polynomial Regressor

In [240]:
from  sklearn.preprocessing  import PolynomialFeatures
poly = PolynomialFeatures(2)
X_train_poly = poly.fit_transform(X_train)
X_test_poly = poly.transform(X_test)
lreg.fit(X_train_poly, y_train)
print(lreg.score(X_train_poly, y_train))
print(lreg.score(X_test_poly, y_test))

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

0.8292634735187432
-30708.50173755525


In [0]:
#from sklearn.preprocessing import PolynomialFeatures
#poly = PolynomialFeatures(2)

In [0]:
#from  sklearn.preprocessing  import PolynomialFeatures
#deg = np.array([1,2,3])

#model =  PolynomialFeatures()
#grid = GridSearchCV(estimator=model, param_grid=dict(degree=deg))
#grid.fit(X_train, y_train)
#print(grid)
# summarize the results of the grid search
#print(grid.best_score_)
#print(grid.best_estimator_.degree)

In [0]:
#print(train_score_list)
#print(test_score_list)

## SVM Regressor - Linear

In [244]:
X= df2.drop(['Freight Cost (USD)'], axis=1)
y=df2[['Freight Cost (USD)']]
X.shape

(8460, 126)

In [0]:
from sklearn.model_selection import train_test_split

X_train_org, X_test_org, y_train, y_test = train_test_split(X,y,random_state = 0)


In [0]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train_org)
X_test = scaler.transform(X_test_org)

In [247]:
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV

parameters = {'C':[0.01,0.1,1,10]}
svr = SVR(kernel='linear')
clf = GridSearchCV(svr, parameters)
clf.fit(X_train,y_train)
clf.best_params_

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
 

GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=SVR(C=1.0, cache_size=200, coef0=0.0, degree=3,
                           epsilon=0.1, gamma='auto_deprecated',
                           kernel='linear', max_iter=-1, shrinking=True,
                           tol=0.001, verbose=False),
             iid='warn', n_jobs=None, param_grid={'C': [0.01, 0.1, 1, 10]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

{'C': 10}

In [248]:
y_train.shape

(6345, 1)

In [249]:
from sklearn.svm import SVR
regressor=SVR(kernel='linear', C=10)
regressor.fit(X_train,y_train)

pred=regressor.predict(X_test)

print(regressor.score(X_test,y_test))

print(r2_score(y_test,pred))

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVR(C=10, cache_size=200, coef0=0.0, degree=3, epsilon=0.1,
    gamma='auto_deprecated', kernel='linear', max_iter=-1, shrinking=True,
    tol=0.001, verbose=False)

0.02525385328080909
0.02525385328080909


## SVM Regressor - RBF Kernel

In [250]:
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV
parameters = { 'C':[0.01,0.1,1,10],'gamma': [0.01,0.1,1,10]}
svrrbfregressor=SVR(kernel='rbf')
clf = GridSearchCV(svrrbfregressor, parameters)

clf.fit(X_train,y_train)
clf.best_params_

#svrrbfpred=svrrbfregressor.predict(X_test)
#print(r2_score(y_test,svrrbfpred))

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
 

GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=SVR(C=1.0, cache_size=200, coef0=0.0, degree=3,
                           epsilon=0.1, gamma='auto_deprecated', kernel='rbf',
                           max_iter=-1, shrinking=True, tol=0.001,
                           verbose=False),
             iid='warn', n_jobs=None,
             param_grid={'C': [0.01, 0.1, 1, 10], 'gamma': [0.01, 0.1, 1, 10]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

{'C': 10, 'gamma': 0.01}

In [251]:
from sklearn.svm import SVR
regressor=SVR(kernel='rbf', gamma=0.01,C=10)
regressor.fit(X_train,y_train)

pred=regressor.predict(X_test)
print(regressor.score(X_test,y_test))

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVR(C=10, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma=0.01,
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

-0.09588375952918904


## Decision trees

In [0]:

from sklearn.model_selection import train_test_split
X= data1.drop(['Freight Cost (USD)'], axis=1)
y=data1[['Freight Cost (USD)']]
X_train_org, X_test_org, y_train, y_test = train_test_split(X,y, test_size=0.25, random_state = 0)

y_train = np.log(y_train)

from sklearn.tree import DecisionTreeRegressor

tree = DecisionTreeRegressor(max_depth=14).fit(X_train, y_train)

# predict on all data
pred_tree = tree.predict(X_test)
pred_tree_train = tree.predict(X_train)

# undo log-transform
y_predicted = np.exp(pred_tree)
price_tree_train = np.exp(pred_tree_train)

In [253]:
from sklearn.metrics import mean_squared_error, r2_score
model_score = tree.score(X_train,y_train)
print("model score", model_score)
print("test score",r2_score(y_test, y_predicted))

model score 0.902322548704032
test score 0.7357951309146666


In [0]:
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.tree import DecisionTreeRegressor

In [0]:
#def deci_model(X, y):
#    gsc = GridSearchCV(
#        estimator=DecisionTreeRegressor(),
#        param_grid={
#           'max_depth': range(12,13,14),
#            
#        },
#        cv=5, scoring='neg_mean_squared_error', verbose=0, n_jobs=-1)
#    
#    grid_result = gsc.fit(X, y)
#    best_params = grid_result.best_params_
#    
#   dec = DecisionTreeRegressor(max_depth=best_params["max_depth"], random_state=0)# Perform K-Fold CV
#    scores = cross_val_score(dec, X, y, cv=10, scoring='neg_mean_absolute_error')

#    return scores,dec,best_params

In [0]:
#scores,rfr,bestparam=deci_model(X_train,y_train)

In [0]:
#rfr

In [0]:
#bestparam

In [0]:
feature_names=X.columns

In [260]:
# Important features
%matplotlib inline
def plot_feature_importances_cancer(model):
    n_features = X_train.shape[1]
    plt.barh(range(n_features), model.feature_importances_, align='center')
    plt.yticks(np.arange(n_features), feature_names)
    plt.xlabel("Feature importance")
    plt.ylabel("Feature")
    plt.ylim(-1, n_features)

plot_feature_importances_cancer(tree)

In [0]:
#r2_score(y_test,y_pred_random)

In [0]:
#print(grid_cv_dtm.best_score_)
#print(grid_cv_dtm.best_params_)

In [0]:
#dtm.score(X_test,y_test)
#dtm.score(X_test,y_test)

In [0]:
# Decision Tree is giving better results to our model compared to other models

## Random Forest

In [0]:
from sklearn.model_selection import cross_val_score

In [0]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(n_estimators = 1000, random_state = 42)
rf.fit(X_train, y_train)
predictions = rf.predict(X_test)


D:\Anaconda\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


In [0]:
#errors = abs(predictions - y_test)
#round(np.mean(errors))
#mape = 100 * (errors / y_test)
#accuracy = 100 - np.mean(mape)

In [0]:
# Perform Grid-Search
def rfr_model(X, y):
    gsc = GridSearchCV(
        estimator=RandomForestRegressor(),
        param_grid={
            'max_depth': range(3,7),
            'n_estimators': (10, 50, 100, 1000),
        },
        cv=5, scoring='neg_mean_squared_error', verbose=0, n_jobs=-1)
    
    grid_result = gsc.fit(X, y)
    best_params = grid_result.best_params_
    
    rfr = RandomForestRegressor(max_depth=best_params["max_depth"], n_estimators=best_params["n_estimators"], random_state=False, verbose=False)# Perform K-Fold CV
    scores = cross_val_score(rfr, X, y, cv=10, scoring='neg_mean_absolute_error')

    return scores,rfr,best_params

In [0]:
scores,rfr,bestparam=rfr_model(X_train,y_train)

D:\Anaconda\lib\site-packages\sklearn\model_selection\_search.py:740: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)
D:\Anaconda\lib\site-packages\sklearn\model_selection\_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
D:\Anaconda\lib\site-packages\sklearn\model_selection\_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
D:\Anaconda\lib\site-packages\sklearn\model_selection\_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the 

In [0]:
rfr

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=6,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
           oob_score=False, random_state=False, verbose=False,
           warm_start=False)

In [0]:
bestparam

{'max_depth': 6, 'n_estimators': 100}

In [0]:
rfrd= RandomForestRegressor(max_depth = 6, n_estimators= 1000,random_state=0)
rfrd.fit(X_train, y_train)
y_pred_random= rfrd.predict(X_test)



D:\Anaconda\lib\site-packages\ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


In [0]:
r2_score(y_test,y_pred_random)

-0.5265491195850878

In [0]:
scores = cross_val_score(rfr, X_train, y_train, cv=10, scoring='neg_mean_absolute_error')

D:\Anaconda\lib\site-packages\sklearn\model_selection\_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
D:\Anaconda\lib\site-packages\sklearn\model_selection\_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
D:\Anaconda\lib\site-packages\sklearn\model_selection\_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
D:\Anaconda\lib\site-packages\sklearn\model_selection\_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change

In [0]:
scores

array([-0.51428053, -0.48735748, -0.52709997, -0.51023652, -0.52504857,
       -0.53102232, -0.53328563, -0.53366956, -0.50813704, -0.50464395])

In [0]:
# As Expected Random Forest is not the best model due to the datasets sparseness

# Final Comments: 
The data is too sparse to fit in any linear models and as expected the decision tree model can be the best case. 
The random Forest fails since in the bagging method there may be imbalance in data which can cause such models to poorly fit.
Last but not the least. The data is highly multi dimensional which makes it tough for plotting. 

The best model is Decision Tree.

# PROJECT 2


## Bagging

### Decision Tree

In [0]:
# bagging 

# decision tree (Best Model)
X= data1.drop(['Freight Cost (USD)'], axis=1)
y=data1[['Freight Cost (USD)']]
X_train_org, X_test_org, y_train, y_test = train_test_split(X,y, test_size=0.25, random_state = 0)
y_train = np.log(y_train)

In [0]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor

dt_reg = DecisionTreeRegressor(random_state=0)
bag_reg = BaggingRegressor(dt_reg, n_estimators=500, max_samples=100, bootstrap=True, random_state=0, oob_score=True)

In [267]:
bag_reg.fit(X_train, y_train)
y_pred = bag_reg.predict(X_test)

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/bagging.py:398: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return column_or_1d(y, warn=True)


BaggingRegressor(base_estimator=DecisionTreeRegressor(criterion='mse',
                                                      max_depth=None,
                                                      max_features=None,
                                                      max_leaf_nodes=None,
                                                      min_impurity_decrease=0.0,
                                                      min_impurity_split=None,
                                                      min_samples_leaf=1,
                                                      min_samples_split=2,
                                                      min_weight_fraction_leaf=0.0,
                                                      presort=False,
                                                      random_state=0,
                                                      splitter='best'),
                 bootstrap=True, bootstrap_features=False, max_features=1.0,
                 max_samples=1

In [268]:
# decision tree with bagging
from sklearn.metrics import mean_squared_error, r2_score
model_score = bag_reg.score(X_train,y_train)
print("model score", model_score)
print("test score",r2_score(y_test, y_pred))

model score 0.6564577958780646
test score -0.5265589035168465


In [269]:
dt_reg.fit(X_train, y_train)
model_score = dt_reg.score(X_train,y_train)


DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
                      max_leaf_nodes=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      presort=False, random_state=0, splitter='best')

From this comparison we can see that bagging does not help us get a more robust model. 
The variance in the model score and the predicted score in the original decision tree is increased further with bagging to give us a less robust model

### SVR (Linear)

In [273]:
# Bagging model 2

X= df2.drop(['Freight Cost (USD)'], axis=1)
y=df2[['Freight Cost (USD)']]
X.shape

from sklearn.model_selection import train_test_split
X_train_org, X_test_org, y_train, y_test = train_test_split(X,y,random_state = 0)

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_scl = scaler.fit_transform(X_train_org)
X_test_scl = scaler.transform(X_test_org)



(8460, 126)

In [0]:
# SVR (Linear) with bagging
from sklearn.svm import SVR

svr = SVR(kernel='linear', C = 10)

bag_svr = BaggingRegressor(svr, n_estimators=500, max_samples=100, bootstrap=True, random_state=0, oob_score=True)


In [275]:
bag_svr.fit(X_train_scl, y_train)
y_pred_svr = bag_svr.predict(X_test_scl)

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/bagging.py:398: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return column_or_1d(y, warn=True)


BaggingRegressor(base_estimator=SVR(C=10, cache_size=200, coef0=0.0, degree=3,
                                    epsilon=0.1, gamma='auto_deprecated',
                                    kernel='linear', max_iter=-1,
                                    shrinking=True, tol=0.001, verbose=False),
                 bootstrap=True, bootstrap_features=False, max_features=1.0,
                 max_samples=100, n_estimators=500, n_jobs=None, oob_score=True,
                 random_state=0, verbose=0, warm_start=False)

In [276]:
from sklearn.metrics import mean_squared_error, r2_score
model_score_svr = bag_svr.score(X_train_scl,y_train)
print("model score", model_score_svr)
print("test score",r2_score(y_test, y_pred_svr))

model score -0.03515697477938451
test score -0.03947376869626229


## Pasting

### Decision Tree

In [0]:
# Pasting

dt_reg = DecisionTreeRegressor(random_state=0)
bag_reg = BaggingRegressor(dt_reg, n_estimators=500, max_samples=100, bootstrap=False, random_state=0)

In [278]:
bag_reg.fit(X_train, y_train)
y_pred = bag_reg.predict(X_test)

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/bagging.py:398: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return column_or_1d(y, warn=True)


BaggingRegressor(base_estimator=DecisionTreeRegressor(criterion='mse',
                                                      max_depth=None,
                                                      max_features=None,
                                                      max_leaf_nodes=None,
                                                      min_impurity_decrease=0.0,
                                                      min_impurity_split=None,
                                                      min_samples_leaf=1,
                                                      min_samples_split=2,
                                                      min_weight_fraction_leaf=0.0,
                                                      presort=False,
                                                      random_state=0,
                                                      splitter='best'),
                 bootstrap=False, bootstrap_features=False, max_features=1.0,
                 max_samples=

In [279]:
# decision tree with pasting
from sklearn.metrics import mean_squared_error, r2_score
model_score = bag_reg.score(X_train,y_train)
print("model score", model_score)
print("test score",r2_score(y_test, y_pred))

model score 0.4664648769075367
test score 0.4728847039558607


### SVR (Linear)

In [0]:
svr = SVR(kernel='linear', C = 10)

bag_svr = BaggingRegressor(svr, n_estimators=500, max_samples=100, bootstrap=False, random_state=0)


In [281]:
bag_svr.fit(X_train_scl, y_train)
y_pred_svr = bag_svr.predict(X_test_scl)

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/bagging.py:398: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return column_or_1d(y, warn=True)


BaggingRegressor(base_estimator=SVR(C=10, cache_size=200, coef0=0.0, degree=3,
                                    epsilon=0.1, gamma='auto_deprecated',
                                    kernel='linear', max_iter=-1,
                                    shrinking=True, tol=0.001, verbose=False),
                 bootstrap=False, bootstrap_features=False, max_features=1.0,
                 max_samples=100, n_estimators=500, n_jobs=None,
                 oob_score=False, random_state=0, verbose=0, warm_start=False)

In [282]:
model_score_svr = bag_svr.score(X_train_scl,y_train)
print("model score", model_score_svr)
print("test score",r2_score(y_test, y_pred_svr))

model score -0.03679666412774729
test score -0.041387205109187475


## Adaboosting

### Decision Tree

In [283]:
from sklearn.ensemble import AdaBoostRegressor


ada_reg1 = AdaBoostRegressor(DecisionTreeRegressor(max_depth=4),
                          n_estimators=300, random_state=0)


ada_reg1.fit(X_train_scl, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


AdaBoostRegressor(base_estimator=DecisionTreeRegressor(criterion='mse',
                                                       max_depth=4,
                                                       max_features=None,
                                                       max_leaf_nodes=None,
                                                       min_impurity_decrease=0.0,
                                                       min_impurity_split=None,
                                                       min_samples_leaf=1,
                                                       min_samples_split=2,
                                                       min_weight_fraction_leaf=0.0,
                                                       presort=False,
                                                       random_state=None,
                                                       splitter='best'),
                  learning_rate=1.0, loss='linear', n_estimators=300,
                  random_

In [0]:
y_pred_ada1 = ada_reg1.predict(X_test_scl)

In [285]:
model_score_ada1 = ada_reg1.score(X_train_scl,y_train)
print("model score", model_score_ada1)
print("test score",r2_score(y_test, y_pred_ada1))

model score 0.5346525489618021
test score 0.4358524701111901


### SVR (Linear)

In [286]:
ada_reg2 = AdaBoostRegressor(svr, n_estimators=500, random_state=0)
ada_reg2.fit(X_train_scl, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


AdaBoostRegressor(base_estimator=SVR(C=10, cache_size=200, coef0=0.0, degree=3,
                                     epsilon=0.1, gamma='auto_deprecated',
                                     kernel='linear', max_iter=-1,
                                     shrinking=True, tol=0.001, verbose=False),
                  learning_rate=1.0, loss='linear', n_estimators=500,
                  random_state=0)

In [0]:
 y_pred_ada2 = ada_reg2.predict(X_test_scl)

In [288]:
model_score_ada2 = ada_reg2.score(X_train_scl,y_train)
print("model score", model_score_ada2)
print("test score",r2_score(y_test, y_pred_ada2))

model score -0.5444138298596481
test score -0.9943088374855729


## Gradient Boosting

In [0]:
from sklearn.ensemble import GradientBoostingRegressor



In [290]:
gbrt = GradientBoostingRegressor(max_depth=2, n_estimators=3, learning_rate=1.0, random_state=42)
gbrt.fit(X_train_scl, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/gradient_boosting.py:1450: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
                          learning_rate=1.0, loss='ls', max_depth=2,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=3,
                          n_iter_no_change=None, presort='auto',
                          random_state=42, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

In [0]:
 y_pred_gd = gbrt.predict(X_test_scl)

In [292]:
model_score_gd = gbrt.score(X_train_scl,y_train)
print("model score", model_score_gd)
print("test score",r2_score(y_test, y_pred_gd))

model score 0.46075640434809784
test score 0.3590569180567422


In [293]:
gbrt_slow = GradientBoostingRegressor(max_depth=2, n_estimators=200, learning_rate=0.1, random_state=42)
gbrt_slow.fit(X_train_scl, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/gradient_boosting.py:1450: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
                          learning_rate=0.1, loss='ls', max_depth=2,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=200,
                          n_iter_no_change=None, presort='auto',
                          random_state=42, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

In [0]:
 y_pred_gds = gbrt_slow.predict(X_test_scl)

In [295]:
model_score_gds = gbrt_slow.score(X_train_scl,y_train)
print("model score", model_score_gds)
print("test score",r2_score(y_test, y_pred_gds))

model score 0.7337009098463974
test score 0.6416182606147434


## PCA

In [0]:
from sklearn.decomposition import PCA

In [0]:
pca = PCA(n_components=0.95)
X_train_reduced = pca.fit_transform(X_train_scl)
X_test_reduced = pca.transform(X_test_scl)

In [0]:
X_train_red_scl = scaler.fit_transform(X_train_reduced)
X_test_red_scl = scaler.transform(X_test_reduced)

### KNN

In [299]:
from sklearn import neighbors
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import GridSearchCV

params = {'n_neighbors':[2,3,4,5,6,7,8,9,11,12,13,14,15,16]}
 
knn = neighbors.KNeighborsRegressor()
model = GridSearchCV(knn, params, cv=5)
model.fit(X_train_red_scl,y_train)
model.best_params_


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=KNeighborsRegressor(algorithm='auto', leaf_size=30,
                                           metric='minkowski',
                                           metric_params=None, n_jobs=None,
                                           n_neighbors=5, p=2,
                                           weights='uniform'),
             iid='warn', n_jobs=None,
             param_grid={'n_neighbors': [2, 3, 4, 5, 6, 7, 8, 9, 11, 12, 13, 14,
                                         15, 16]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

{'n_neighbors': 2}

In [300]:
knn = neighbors.KNeighborsRegressor(2)
knn.fit(X_train_red_scl, y_train)
y_pred_knn=knn.predict(X_test_red_scl)
print("KNN train score ",knn.score(X_train_red_scl, y_train))
#print("Mean squared error ",mean_squared_error(y_test,y_pred_knn))
print("KNN test score ",knn.score(X_test_red_scl, y_test))

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
                    metric_params=None, n_jobs=None, n_neighbors=2, p=2,
                    weights='uniform')

KNN train score  0.8540826869901061
KNN test score  0.4616447232253879


### Linear Regression

In [301]:
from sklearn.linear_model import LinearRegression

lreg = LinearRegression()
lreg.fit(X_train_red_scl, y_train)
print(lreg.score(X_train_red_scl, y_train))
print(lreg.score(X_test_red_scl, y_test))

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

0.3099910069800632
0.18218469836381246


### Ridge Regression

In [302]:
from  sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV

alphas = np.array([1,0.1,0.01,0.001,0.0001,0])

model = Ridge()
grid = GridSearchCV(estimator=model, param_grid=dict(alpha=alphas))
grid.fit(X_train_red_scl, y_train)
print(grid)
# summarize the results of the grid search
print(grid.best_score_)
print(grid.best_estimator_.alpha)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=Ridge(alpha=1.0, copy_X=True, fit_intercept=True,
                             max_iter=None, normalize=False, random_state=None,
                             solver='auto', tol=0.001),
             iid='warn', n_jobs=None,
             param_grid={'alpha': array([1.e+00, 1.e-01, 1.e-02, 1.e-03, 1.e-04, 0.e+00])},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=Ridge(alpha=1.0, copy_X=True, fit_intercept=True,
                             max_iter=None, normalize=False, random_state=None,
                             solver='auto', tol=0.001),
             iid='warn', n_jobs=None,
             param_grid={'alpha': array([1.e+00, 1.e-01, 1.e-02, 1.e-03, 1.e-04, 0.e+00])},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)
-8.93998914294161
1.0


In [303]:
ridge = Ridge(alpha=1.0)
ridge.fit(X_train,y_train)
print(ridge.score(X_train,y_train))
print(ridge.score(X_test,y_test))

Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
      normalize=False, random_state=None, solver='auto', tol=0.001)

0.34454820766929284
-0.14671385522504954


### Lasso Regression

In [304]:
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV

alphas = np.array([1,0.1,0.01,0.001,0.0001,0])

model = Lasso()
grid = GridSearchCV(estimator=model, param_grid=dict(alpha=alphas))
grid.fit(X_train_red_scl, y_train)
print(grid)
# summarize the results of the grid search
print(grid.best_score_)
print(grid.best_estimator_.alpha)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 163266206034.403, tolerance: 147815657.8999031
  positive)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 329704830897.5448, tolerance: 136043465.5396917
  positive)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3712532493.187378, tolerance:

GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=Lasso(alpha=1.0, copy_X=True, fit_intercept=True,
                             max_iter=1000, normalize=False, positive=False,
                             precompute=False, random_state=None,
                             selection='cyclic', tol=0.0001, warm_start=False),
             iid='warn', n_jobs=None,
             param_grid={'alpha': array([1.e+00, 1.e-01, 1.e-02, 1.e-03, 1.e-04, 0.e+00])},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=Lasso(alpha=1.0, copy_X=True, fit_intercept=True,
                             max_iter=1000, normalize=False, positive=False,
                             precompute=False, random_state=None,
                             selection='cyclic', tol=0.0001, warm_start=False),
             iid='warn', n_jobs=None,
             param_grid={'alpha': array([1.e+00, 1.e-01, 1.e-02, 1.e-03, 1.e-04, 0.e+00])},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)
-5.552307414659991
1.0


In [305]:
from sklearn.linear_model import Lasso
lasso = Lasso(alpha=1.0)
lasso.fit(X_train_red_scl,y_train)
print(lasso.score(X_train_red_scl,y_train))
print(lasso.score(X_test_red_scl,y_test))

Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=1000,
      normalize=False, positive=False, precompute=False, random_state=None,
      selection='cyclic', tol=0.0001, warm_start=False)

0.3099907496914883
0.18240592548569778


### Polynomial Regression

In [306]:
from  sklearn.preprocessing  import PolynomialFeatures
poly = PolynomialFeatures(2)
X_train_poly = poly.fit_transform(X_train_red_scl)
X_test_poly = poly.transform(X_test_red_scl)
lreg.fit(X_train_poly, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [307]:
print(lreg.score(X_train_poly, y_train))
print(lreg.score(X_test_poly, y_test))

0.8395997244723046
-3.316678778745732e+22


### SVM Regressor - Linear

In [308]:
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV

parameters = {'C':[0.01,0.1,1,10]}
svr = SVR(kernel='linear')
clf = GridSearchCV(svr, parameters)
clf.fit(X_train_red_scl, y_train)
clf.best_params_

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
 

GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=SVR(C=1.0, cache_size=200, coef0=0.0, degree=3,
                           epsilon=0.1, gamma='auto_deprecated',
                           kernel='linear', max_iter=-1, shrinking=True,
                           tol=0.001, verbose=False),
             iid='warn', n_jobs=None, param_grid={'C': [0.01, 0.1, 1, 10]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

{'C': 10}

In [309]:
from sklearn.svm import SVR

regressor=SVR(kernel='linear', C=10)
regressor.fit(X_train_red_scl,y_train)

pred=regressor.predict(X_test_red_scl)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVR(C=10, cache_size=200, coef0=0.0, degree=3, epsilon=0.1,
    gamma='auto_deprecated', kernel='linear', max_iter=-1, shrinking=True,
    tol=0.001, verbose=False)

In [310]:
print(regressor.score(X_test_red_scl,y_test))


0.15893778504603095


### SVM Regressor - RBF Kernel

In [311]:
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV
parameters = { 'C':[0.01,0.1,1,10],'gamma': [0.01,0.1,1,10]}
svrrbfregressor=SVR(kernel='rbf')
clf = GridSearchCV(svrrbfregressor, parameters)

clf.fit(X_train_red_scl,y_train)
clf.best_params_

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
 

GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=SVR(C=1.0, cache_size=200, coef0=0.0, degree=3,
                           epsilon=0.1, gamma='auto_deprecated', kernel='rbf',
                           max_iter=-1, shrinking=True, tol=0.001,
                           verbose=False),
             iid='warn', n_jobs=None,
             param_grid={'C': [0.01, 0.1, 1, 10], 'gamma': [0.01, 0.1, 1, 10]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

{'C': 10, 'gamma': 0.01}

In [312]:
from sklearn.svm import SVR
regressor=SVR(kernel='rbf', gamma=0.01,C=10)
regressor.fit(X_train_red_scl,y_train)



/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVR(C=10, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma=0.01,
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [313]:
pred=regressor.predict(X_test_red_scl)
print(regressor.score(X_test_red_scl,y_test))

-0.10096993427272927


### Decision Tree

In [314]:
from sklearn.tree import DecisionTreeRegressor

param_grid2 = {'min_samples_split' : range(10,500,20),'max_depth': range(1,20,2)}

grid2 = GridSearchCV(DecisionTreeRegressor(random_state = 0), param_grid2, cv = 5, return_train_score= True, iid = False)

grid2.fit(X_train_red_scl, y_train)

GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=DecisionTreeRegressor(criterion='mse', max_depth=None,
                                             max_features=None,
                                             max_leaf_nodes=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             presort=False, random_state=0,
                                             splitter='best'),
             iid=False, n_jobs=None,
             param_grid={'max_depth': range(1, 20, 2),
                         'min_samples_split': range(10, 500, 20)},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
             scorin

In [315]:
print(grid2.best_score_)
print(grid2.best_params_)

0.27613317146609573
{'max_depth': 17, 'min_samples_split': 50}


In [316]:
grid2.score(X_test_red_scl, y_test)

0.11370683607380196

## Comparison Table

In [0]:
comp_table = [['KNN', 0.50128, 0.46164], ['Linear Regression', 0.18496, 0.18218], ['Ridge Regression', 0.18138, -0.14671], ['Lasso Regression', 0.17259, 0.18240], ['Polynomial Regression', -10452.32885, -3.31667], ['SVM Regressor - Linear', 0.02525, 0.15893], ['SVM Regressor - RBF Kernel', -0.09588, -0.10096], ['Decision Tree', 0.72918, 0.11370]]

df_comp = pd.DataFrame(comp_table, columns = ['Model', 'Test score without PCA', 'Test score after PCA'])

In [0]:
df_comp

Model  Test score without PCA  Test score after PCA
0                         KNN                 0.50128               0.46164
1           Linear Regression                 0.18496               0.18218
2            Ridge Regression                 0.18138              -0.14671
3            Lasso Regression                 0.17259               0.18240
4       Polynomial Regression            -10452.32885              -3.31667
5      SVM Regressor - Linear                 0.02525               0.15893
6  SVM Regressor - RBF Kernel                -0.09588              -0.10096
7               Decision Tree                 0.72918               0.11370

#### The best model we got before PCA was from using the Decision Tree algorithm with a test accuracy of 72.91%. But the same decision tree after PCA has only 11.37% accuracy. This is due to the data being too sparse to fit in any linear models. I believe that in this situation a model without employing PCA will give us more robust solutions. 

## ANN - Regression

In [323]:
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold


NN_model = Sequential()

# The Input Layer :
NN_model.add(Dense(256, kernel_initializer='normal',input_dim = X_train_scl.shape[1], activation='relu'))

# The Hidden Layers :
NN_model.add(Dense(128, kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(64, kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(32, kernel_initializer='normal',activation='relu'))

# The Output Layer :
NN_model.add(Dense(1, kernel_initializer='normal',activation='linear'))

# Compile the network :
NN_model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])
NN_model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_16 (Dense)             (None, 256)               32512     
_________________________________________________________________
dense_17 (Dense)             (None, 128)               32896     
_________________________________________________________________
dense_18 (Dense)             (None, 64)                8256      
_________________________________________________________________
dense_19 (Dense)             (None, 32)                2080      
_________________________________________________________________
dense_20 (Dense)             (None, 1)                 33        
Total params: 75,777
Trainable params: 75,777
Non-trainable params: 0
_________________________________________________________________


In [0]:
from keras.callbacks import ModelCheckpoint

checkpoint_name = 'Weights-{epoch:03d}--{val_loss:.5f}.hdf5' 
checkpoint = ModelCheckpoint(checkpoint_name, monitor='val_loss', verbose = 1, save_best_only = True, mode ='auto')
callbacks_list = [checkpoint]

In [325]:
NN_model.fit(X_train_scl, y_train, validation_data=(X_test_scl, y_test), epochs=25, batch_size=32, callbacks=callbacks_list)


Train on 6345 samples, validate on 2115 samples
Epoch 1/25
6345/6345 [==============================] - 2s 247us/step - loss: 10193.2158 - mean_absolute_error: 10193.2158 - val_loss: 7944.3911 - val_mean_absolute_error: 7944.3911

Epoch 00001: val_loss improved from inf to 7944.39115, saving model to Weights-001--7944.39115.hdf5
Epoch 2/25
6345/6345 [==============================] - 1s 179us/step - loss: 7708.8568 - mean_absolute_error: 7708.8568 - val_loss: 7383.1564 - val_mean_absolute_error: 7383.1564

Epoch 00002: val_loss improved from 7944.39115 to 7383.15637, saving model to Weights-002--7383.15637.hdf5
Epoch 3/25
6345/6345 [==============================] - 1s 178us/step - loss: 7322.0160 - mean_absolute_error: 7322.0160 - val_loss: 7209.6324 - val_mean_absolute_error: 7209.6324

Epoch 00003: val_loss improved from 7383.15637 to 7209.63240, saving model to Weights-003--7209.63240.hdf5
Epoch 4/25
6345/6345 [==============================] - 1s 175us/step - loss: 7124.0931 - mea

In [326]:
# evaluate the model
print(NN_model.evaluate(X_train_scl, y_train))
print(NN_model.evaluate(X_test_scl, y_test))

6345/6345 [==============================] - 0s 48us/step
[4614.079941567856, 4614.079941567856]
2115/2115 [==============================] - 0s 42us/step
[5279.606508338874, 5279.606508338874]
